In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.12.1 True
2.25.2
10.2
GCC 7.3


In [2]:
import mmcv
import matplotlib.pyplot as plt

In [3]:
import mmcv
from mmcv import Config
cfg = Config.fromfile('/home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2/cascade_rcnn_lagfart.py')

In [4]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
#cfg.dataset_type = 'KittiTinyDataset'

# modify num classes of the model in box head
#cfg.model.roi_head.bbox_head.num_classes = 1
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
#cfg.load_from = '/home/erik/Riksarkivet/Projects/fastighet/models/checkpoints/cascade_rcnn_r101_caffe_fpn_1x_coco_bbox_mAP-0.423_20200504_175649-cab8dbd5.pth'

# Set up working dir to save files and logs.
cfg.work_dir = '/home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 200

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'bbox'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 4

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = 'cuda'


In [5]:
cfg.dump('/home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2/cascade_rcnn_lagfart.py')

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import os


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [4]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
checkpoint = '/home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2/latest.pth'
model = init_detector(cfg, checkpoint, device='cuda:0')

load checkpoint from local path: /home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2/latest.pth


In [12]:
from glob import glob
from random import sample
from pathlib import Path

batches = glob(os.path.join('/media/erik/T7/Data/Lagfarter/cropped_imgs_ind_batches', '**'))
batches_sample = sample(batches, 200)

for i, batch in enumerate(batches_sample):
    imgs = glob(os.path.join(batch, '**'))
    imgs_sample = sample(imgs, 3)
    
    for img_p in imgs_sample:
        img = mmcv.imread(img_p)
        result = inference_detector(model, img)

        out = Path(img_p).name
        out_p = os.path.join('/home/erik/Riksarkivet/Projects/lagfarter/output/eval_det_95', out)
        model.show_result(img, result, out_file=out_p, score_thr=0.95, bbox_color=(0,1,2,3))

    print(i)




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [ ]:
im = mmcv.imread('/home/erik/Riksarkivet/Projects/lagfarter/data/cropped_imgs_ind_batches/H0000805/SE_HLA1040187_CIab_2_000037.tiff')

#model.cfg = cfg
result = inference_detector(model, im)
show_result_pyplot(model, im, result, score_thr=0.3)
"""out = Path(img).name
out = out.split('-')[1]
out_p = os.path.join('/content/drive/MyDrive/Riksarkivet/Fastighet/testmodel_mmdet/inference_on_eval_set', out)
model.show_result(im, result, out_file=out_p)"""

In [ ]:
cfg.dump('/home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2/cascade_rcnn_lagfart.py')

Error: Kernel is dead

In [ ]:
from glob import glob
from pathlib import Path
import os
import mmcv

path_to_imgs = '/home/erik/Riksarkivet/Projects/fastighet/data/testbatch/10001001'
imgs = glob(os.path.join(path_to_imgs, '**'))
imgs.sort()

for i, img in enumerate(imgs):
  im = mmcv.imread(img)
  
  

  #model.cfg = cfg
  #result = inference_detector(model, im)
  #show_result_pyplot(model, im, result, score_thr=0.6)
  #out = Path(img).name
  #out_p = os.path.join('/home/erik/Riksarkivet/Projects/fastighet/data/testbatch/results', out)
  #model.show_result(im, result, out_file=out_p)
  #if i % 20 == 0:
    #print(i)

In [ ]:
for r in result:
    print(r)